In [1]:
from torchnlp.datasets import penn_treebank_dataset
import torch
from torchnlp.samplers import BPTTBatchSampler
from torch.utils.data import DataLoader
from rsm_samplers import MNISTSequenceSampler, ptb_pred_sequence_collate
from ptb_lstm import LSTMModel
from lang_util import Corpus
from torchvision import datasets, transforms
import torch.nn.functional as F
import torchvision.utils as vutils
from torch.nn import CrossEntropyLoss
from importlib import reload 
from torch.utils.data import Sampler, BatchSampler
import rsm
from functools import reduce, partial
import matplotlib.pyplot as plt

In [11]:
corpus = Corpus('/Users/jgordon/nta/datasets/PTB')

In [70]:
import rsm_samplers
import rsm
import lang_util
reload(rsm_samplers)
reload(rsm)
reload(lang_util)

BS = 10
SEQL = 2
VS = 10000
EMB_DIM = 28

bwe = lang_util.BitwiseWordEmbedding()

vector_dict = {}
for i in range(VS):
    random_binary = torch.FloatTensor(EMB_DIM).uniform_() > 0.85
    vector_dict[i] = random_binary

model = rsm.RSMLayer(d_in=EMB_DIM, d_out=EMB_DIM, embed_dim=EMB_DIM)
collate_fn = partial(rsm_samplers.ptb_pred_sequence_collate, vector_dict=bwe.embedding_dict)

sampler = rsm_samplers.PTBSequenceSampler(corpus.train, batch_size=BS, seq_length=SEQL)
loader = DataLoader(corpus.train,
                       batch_sampler=sampler,
                       collate_fn=collate_fn)


In [71]:
next(iter(sampler))

(tensor([[ 365,   27,   27,  108, 6220, 1015,  238,  108,  805, 3509],
         [  27,   27, 2478,   32, 2577,  525,  220,  366,   32,  219]]),
 tensor([[  27,   27, 2478,   32, 2577,  525,  220,  366,   32,  219],
         [ 453,   42,  229,  237,   42,   24,   26,   48, 3818,  152]]))

In [67]:

# batch = next(iter(loader))
# inputs, target, pred_target = batch
# print(inputs.size())
# hidden = model.init_hidden(BS)
# x_a, hidden, x_b = model(inputs, hidden)
# print(x_a.size())

torch.Size([1, 10, 28])
torch.Size([1, 10, 28])


In [30]:
import rsm_samplers
reload(rsm_samplers)

from torch.utils.data import DataLoader, BatchSampler

dataset = datasets.MNIST("~/nta/datasets", download=True,
                                               transform=transforms.Compose([
                                                   transforms.ToTensor(),
                                                   transforms.Normalize((0.1307,), (0.3081,))
                                               ]),)

sl = 12
bs = 10
sampler = rsm_samplers.MNISTSequenceSampler(dataset, batch_size=bs, sequences=[[0,1,2,3],[4,5,6,7],[9,8,7,6]], randomize_sequences=True)
batch_sampler = BatchSampler(sampler, batch_size=sl * bs + 1, drop_last=True)

collate_fn = partial(rsm_samplers.pred_sequence_collate, 
                     bsz=bs,
                     seq_length=sl)
loader = DataLoader(dataset,
                               batch_sampler=batch_sampler,
                               collate_fn=collate_fn)
model = rsm.RSMLayer()

In [33]:
class BitwiseWordEmbedding(object):

    def __init__(self, vocab_size=10000, dim=28):
        self.vocab_size = vocab_size
        self.embedding_dict = {}
        self.dim = dim

    def generate_embeddings(self):
        for i in range(self.vocab_size):
            self.embedding_dict[i] = self.embed(i)

    def embed(self, i):
        first = "{0:b}".format(i).zfill(self.dim // 2)
        return first + self.inverse(first)

    def inverse(self, binstr):
        return ''.join('1' if x == '0' else '0' for x in binstr)

bwe = BitwiseWordEmbedding()

bwe.embed(9381)

'1001001010010101101101011010'